<a href="https://colab.research.google.com/github/Correa-D-Lucas/bike_share_service_2022/blob/main/bike_share_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Authenticate to GCP

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
project_id = 'citric-inkwell-375211'

# 2. Connect to BigQuery API

In [3]:
from google.cloud import bigquery

In [4]:
client = bigquery.Client(project=project_id)

# 3. Acess the Dataset

In [5]:
dataset_ref = client.dataset("bike_share", project = 'lucas-project-373901')

In [6]:
dataset = client.get_dataset(dataset_ref)

# 4. List Tables

In [7]:
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

2022_01
2022_02
2022_03
2022_04
2022_05
2022_06
2022_07
2022_08
2022_09
2022_10
2022_11
2022_12
2022_all_year


# 5. Check Table Schema

In [8]:
table_ref = dataset_ref.table('2022_all_year')
table = client.get_table(table_ref)

In [9]:
table.schema

[SchemaField('ride_id', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('rideable_type', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('started_at', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('ended_at', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('start_station_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('start_station_id', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('end_station_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('end_station_id', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('start_lat', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('start_lng', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('end_lat', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('end_lng', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('member_casual', 'STRING', 'NULLABLE', None, None, (), None)]

# 6. Show Table as DataFrame

In [10]:
df = client.list_rows(table).to_dataframe()
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,327DA49716F4B82F,classic_bike,2022-08-18 10:15:23+00:00,2022-08-18 10:39:49+00:00,DuSable Lake Shore Dr & North Blvd,LF-005,DuSable Lake Shore Dr & Monroe St,13300,41.911722,-87.626804,41.880958,-87.616743,casual
1,1382E3C7697FB94C,classic_bike,2022-08-18 10:15:23+00:00,2022-08-18 10:30:29+00:00,Orleans St & Chestnut St (NEXT Apts),620,Ashland Ave & Division St,13061,41.898203,-87.637536,41.903450,-87.667747,member
2,ED1F68D415C6A42C,classic_bike,2022-08-18 10:15:26+00:00,2022-08-18 10:24:26+00:00,Franklin St & Illinois St,RN-,Canal St & Adams St,13011,41.891023,-87.635480,41.879255,-87.639904,casual
3,9EB885B34425A91F,electric_bike,2022-08-18 10:15:31+00:00,2022-08-18 10:40:41+00:00,None,None,None,None,41.890000,-87.640000,41.940000,-87.640000,casual
4,1EBFB9643C35DA35,electric_bike,2022-08-18 10:15:31+00:00,2022-08-18 10:29:52+00:00,None,None,None,None,41.850000,-87.690000,41.890000,-87.690000,member


# 7. Disabling Data Table Display

In [11]:
from google.colab import data_table
data_table.disable_dataframe_formatter()

# 8. Data Cleaning Process

In [12]:
# Importing packages

In [13]:
import pandas as pd
import numpy as pd 
import datetime 
from datetime import timedelta

In [14]:
# Create a column named 'trip_duration'

In [15]:
df['trip_duration'] = df['ended_at'] - df['started_at']
df['trip_duration'].sort_values(ascending = False)

319629     28 days 17:47:15
4716252    25 days 04:17:48
3378968    24 days 21:00:38
1382478    23 days 20:34:04
609537     23 days 18:08:49
                 ...       
2534610   -1 days +21:50:55
2534631   -1 days +21:49:58
2534753   -1 days +21:42:35
1484146   -1 days +21:11:18
5048035   -8 days +19:26:39
Name: trip_duration, Length: 5667717, dtype: timedelta64[ns]

In [17]:
# Noticed negative days - It implies that 'started_at' values were greater than 'ended_at' values - Not realistic. 

# How many rows containing this error? 

In [18]:
error = df[df['started_at'] >= df['ended_at']].index
len(error)

531

In [19]:
# Calculate percentage of error 

In [ ]:
(len(error))/len(df)*100 

0.00936885169107773

*That is about 0.0093%. I am comfortable in filtering them out.*

In [20]:
# Data without data errors

In [21]:
df.drop(error, inplace = True)         

In [22]:
# Checking if still exist no negavite days (in percentage of a hour)

In [23]:
df['trip_duration'] = df['trip_duration'].dt.total_seconds().sort_values(ascending = False)  
df['trip_duration_hr'] = df['trip_duration'] / 3600.0   
df['trip_duration_hr']    

0          0.407222
1          0.251667
2          0.150000
3          0.419444
4          0.239167
             ...   
5667712    0.215278
5667713    0.088056
5667714    0.126111
5667715    0.092222
5667716    0.122778
Name: trip_duration_hr, Length: 5667186, dtype: float64

In [25]:
# (Commented !!!) Converting timedelta64[ns] to HH:MM:SS format ----- It would make the column be an Object (data type), I decided not to use it and work with float64. 

In [24]:
# def sec_to_format(s):
   # h,r = divmod(int(s),3600)
   # m,s = divmod(r,60)
   # return f'{h:03}:{m:02}:{s:02}'

# df['trip_duration'] = [sec_to_format(s) for s in df['trip_duration'].dt.total_seconds()]
# df['trip_duration'].sort_values(ascending = False)                                        

In [ ]:
# Checking data type for 'started_at' column

In [26]:
type(df['started_at'])

pandas.core.series.Series

#### INFO ABOUT method dt.dayofweek
Series.dt.dayofweek
(The day of the week with Monday=0, Sunday=6)

In [27]:
# Create a column to express day of the week

In [28]:
df['day_of_week_num'] = df['started_at'].dt.dayofweek
df['day_of_week_num'].head()

0    3
1    3
2    3
3    3
4    3
Name: day_of_week_num, dtype: int64

In [ ]:
# Create a column with days of the week as string

In [29]:
df['day_of_week_str'] = df.day_of_week_num.replace({0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'saturday', 6: 'Sunday'})

In [30]:
# Concatenate 'start_station_name' with 'end_station_name' to create a new column named "Route"

In [31]:
df["Route"] = df['start_station_name'] + ' to ' + df['end_station_name']

In [32]:
# Checking on the columns I have created

In [33]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,trip_duration_hr,day_of_week_num,day_of_week_str,Route
0,327DA49716F4B82F,classic_bike,2022-08-18 10:15:23+00:00,2022-08-18 10:39:49+00:00,DuSable Lake Shore Dr & North Blvd,LF-005,DuSable Lake Shore Dr & Monroe St,13300,41.911722,-87.626804,41.880958,-87.616743,casual,1466.0,0.407222,3,Thursday,DuSable Lake Shore Dr & North Blvd to DuSable ...
1,1382E3C7697FB94C,classic_bike,2022-08-18 10:15:23+00:00,2022-08-18 10:30:29+00:00,Orleans St & Chestnut St (NEXT Apts),620,Ashland Ave & Division St,13061,41.898203,-87.637536,41.903450,-87.667747,member,906.0,0.251667,3,Thursday,Orleans St & Chestnut St (NEXT Apts) to Ashlan...
2,ED1F68D415C6A42C,classic_bike,2022-08-18 10:15:26+00:00,2022-08-18 10:24:26+00:00,Franklin St & Illinois St,RN-,Canal St & Adams St,13011,41.891023,-87.635480,41.879255,-87.639904,casual,540.0,0.150000,3,Thursday,Franklin St & Illinois St to Canal St & Adams St
3,9EB885B34425A91F,electric_bike,2022-08-18 10:15:31+00:00,2022-08-18 10:40:41+00:00,None,None,None,None,41.890000,-87.640000,41.940000,-87.640000,casual,1510.0,0.419444,3,Thursday,NaN
4,1EBFB9643C35DA35,electric_bike,2022-08-18 10:15:31+00:00,2022-08-18 10:29:52+00:00,None,None,None,None,41.850000,-87.690000,41.890000,-87.690000,member,861.0,0.239167,3,Thursday,NaN


# 9. Analytical Investigation

### Hypothesis: **Member uses bike share service to go to work, while casual riders use it for enjoyment.**

In [47]:
# To get statistical info from dataset

In [35]:
df.describe(include = 'all')

<ipython-input-35-74aa2f970831>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include = 'all')


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,trip_duration_hr,day_of_week_num,day_of_week_str,Route
count,5667186,5667186,5667186,5667186,4834148,4834148,4774650,4774650,5.667186e+06,5.667186e+06,5.661328e+06,5.661328e+06,5667186,5.667186e+06,5.667186e+06,5.667186e+06,5667186,4369052
unique,5667186,3,4745500,4758286,1674,1338,1692,1341,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,7,174372
top,327DA49716F4B82F,electric_bike,2022-05-30 13:05:15+00:00,2022-08-22 12:47:49+00:00,Streeter Dr & Grand Ave,13022,Streeter Dr & Grand Ave,13022,NaN,NaN,NaN,NaN,member,NaN,NaN,NaN,saturday,Streeter Dr & Grand Ave to Streeter Dr & Grand...
freq,1,2888630,9,20,75222,75222,75371,75371,NaN,NaN,NaN,NaN,3345417,NaN,NaN,NaN,916376,12192
first,NaN,NaN,2022-01-01 00:00:05+00:00,2022-01-01 00:01:48+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2022-12-31 23:59:26+00:00,2023-01-02 04:56:45+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.190222e+01,-8.764783e+01,4.190242e+01,-8.764790e+01,NaN,1.166846e+03,3.241238e-01,3.061289e+00,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.626074e-02,2.999933e-02,6.805964e-02,1.083032e-01,NaN,1.056493e+04,2.934704e+00,1.975524e+00,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.164000e+01,-8.784000e+01,0.000000e+00,-8.814000e+01,NaN,1.000000e+00,2.777778e-04,0.000000e+00,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.188103e+01,-8.766154e+01,4.188103e+01,-8.766260e+01,NaN,3.490000e+02,9.694444e-02,1.000000e+00,NaN,NaN


In [36]:
# A few insights from the statistical information

1. No duplicates 
2. eletric bike is the most used. 
3. streeter dr & grand ave most used bike station. 
4. members are more often riding. 
5. mean of trip duration is about 19.2 minutes (0.32% of hour)
6. saturday is the busiest day (usually)
7. most common route = streeter dr & grand ave 

In [37]:
# Max time trip (in minutes)

In [38]:
max(df['trip_duration'])/60     

41387.25

### **Number of Rides and Time of Rides per Day of the Week**

In [39]:
# Average of time trip per days of the week (in minutes)

In [40]:
((df.groupby('day_of_week_str')['trip_duration_hr'].mean())*60).sort_values(ascending = False)   

day_of_week_str
Sunday       24.069652
saturday     23.680691
Friday       19.008261
Monday       18.526242
Thursday     17.166157
Tuesday      16.746811
Wednesday    16.452339
Name: trip_duration_hr, dtype: float64

###### **Sunday** has the **longest trip average**, followed by **Saturday** and **Friday**. (WEEKEND RIDES)

In [48]:
 # Num of trips per days of the week

In [49]:
(df.groupby('day_of_week_str')['ride_id'].count()).sort_values(ascending = False)    

day_of_week_str
saturday     916376
Thursday     841512
Friday       801718
Wednesday    798175
Tuesday      782290
Sunday       776161
Monday       750954
Name: ride_id, dtype: int64

###### Surpringly, **Sunday is the penultimate place in number of rides**. And, as we notice, Sunday is top 1 in hours riding. (Interesting)  

###### **Saturday** is the busiest day, followed by **Thursday** and **Friday**. 

###### Surprisingly, **Thursday** is busiest than** Friday**. (Interesting)

### **Number of Rides and Time of Rides per type of Users**

In [50]:
 # Num of trips per type of customer

In [51]:
(df.groupby(['member_casual'])['ride_id'].count()).sort_values(ascending = False)  

member_casual
member    3345417
casual    2321769
Name: ride_id, dtype: int64

##### **Member** rides **more often** than **Casual.**

In [52]:
# Average time trip per user types

In [53]:
((df.groupby(['member_casual'])['trip_duration_hr'].mean())*60).sort_values(ascending = False) 

member_casual
casual    29.148333
member    12.714858
Name: trip_duration_hr, dtype: float64

##### **Casual** rides **spend more time riding** than **Member.**

In [56]:
 # Average time trip per day of the week per user type (In  minutes)

In [55]:
((df.groupby(['member_casual', 'day_of_week_str'])['trip_duration_hr'].mean())*60).sort_values(ascending = False)

member_casual  day_of_week_str
casual         Sunday             34.060572
               saturday           32.617876
               Monday             29.189675
               Friday             28.046850
               Tuesday            25.825316
               Thursday           25.550397
               Wednesday          24.752198
member         saturday           14.140955
               Sunday             14.032259
               Friday             12.531630
               Thursday           12.293652
               Monday             12.270885
               Tuesday            12.130285
               Wednesday          12.105606
Name: trip_duration_hr, dtype: float64

#### **Insights:** 

###### **Background:** We have seen that **Casual users spend more time riding than member users**. We have also seen that **Sunday is the day which people spend more time riding**. Additionally, **Saturday usually is the day of the week with the greatest number of rides**. 


##### **Casual:** This group has a difference of **~10 minutes riding** between the day with the longest avg ride and the day with the shortest avg ride. Also, **Sunsay is the day this group spend more time riding**(any correlation with Sunday being the day with the longest rides).



##### **Member:** This group has a difference of only **~2 minutes riding** between the day with the longest avg ride and the day with the shortest avg ride. Also, **weekday has difference of less than 1 minute avg ride** (pretty constant). 


In [57]:
# Num of trips per day of the week per user type

In [59]:
(df.groupby(['member_casual', 'day_of_week_str'])['ride_id'].count()).sort_values(ascending = False)

member_casual  day_of_week_str
member         Thursday           532215
               Wednesday          523836
               Tuesday            518584
               Monday             473305
casual         saturday           473130
member         Friday             467051
               saturday           443246
casual         Sunday             388981
member         Sunday             387180
casual         Friday             334667
               Thursday           309297
               Monday             277649
               Wednesday          274339
               Tuesday            263706
Name: ride_id, dtype: int64

### Insights: 

##### **Casual:** Top 3 days of service use: **Saturday**, **Sunday**, and **Friday** (Weekends) 

##### **Member:** Top 5 days of service use: **Thursday**, **Wed**, **Tuesday**, **Monday**, **Friday** (weekdays)


##### Strong Hypothesis proof so far.

In [67]:
# Top 5 most ofren used starting stations 

In [66]:
df['start_station_name'].value_counts()[:5].index.tolist()

['Streeter Dr & Grand Ave',
 'DuSable Lake Shore Dr & Monroe St',
 'DuSable Lake Shore Dr & North Blvd',
 'Michigan Ave & Oak St',
 'Wells St & Concord Ln']

In [68]:
# Top 5 most ofren used ending stations 

In [69]:
df['end_station_name'].value_counts()[:5].index.tolist()

['Streeter Dr & Grand Ave',
 'DuSable Lake Shore Dr & North Blvd',
 'DuSable Lake Shore Dr & Monroe St',
 'Michigan Ave & Oak St',
 'Wells St & Concord Ln']

In [70]:
# Top 5 Routes

In [71]:
df['Route'].value_counts()[:5].index.tolist()

['Streeter Dr & Grand Ave to Streeter Dr & Grand Ave',
 'DuSable Lake Shore Dr & Monroe St to DuSable Lake Shore Dr & Monroe St',
 'Ellis Ave & 60th St to University Ave & 57th St',
 'University Ave & 57th St to Ellis Ave & 60th St',
 'Ellis Ave & 60th St to Ellis Ave & 55th St']

##### **Streeter Dr & Grand Ave** is no doubt the most busiest location. 

##### Why is that important? **Advertising**, **Maintenance**. 

##### I would say our **top 5 important locations** are **"Streeter Dr & Grand Ave"**, "**DuSable Lake Shore Dr & Monroe St"**, and **"Ellis Ave & 60th St"**, **"DuSable Lake Shore Dr & North Blvd"**, and **"University Ave & 57th St"**.

# 10. Export dataframe into BigQuery (for further analysis and later export to Tableau)

In [61]:
import pandas_gbq
pandas_gbq.to_gbq(df, '2022_bike_share.dataframe_2022', project_id= 'citric-inkwell-375211')

100%|██████████| 1/1 [00:00<00:00, 950.44it/s]


# 11. Questions for Work in the Future:

##### Investigate how riders have days of using the service for only one ride. Is it a problem with the checkout system? 

##### Why would casual riders buy Cyclistic annual memberships?

##### How can Cyclists use digital media to influence casual riders to become members?